In [1]:
import logging
import math
import os
import sys
import time
from typing import Optional

# import argparse
import click
import gtfparse

In [2]:
version = 1.5
CLASS_COLUMN_USED = [0, 1, 2, 3, 5, 6, 7, 28, 30, 31]
CLASS_COLUMN_NAME = [
    "isoform",
    "chrom",
    "strand",
    "length",
    "structural_category",
    "associated_gene",
    "associated_transcript",
    "ORF_length",
    "CDS_start",
    "CDS_end",
]

In [3]:
annotation_file = "/home/milo/workspace/Homo_sapiens_GRCh38_Ensembl_86.gff3"

In [4]:
gtfparse.__version__

'1.4.1'

In [5]:
annot_df = gtfparse.parse_gtf(annotation_file)

INFO:gtfparse:Reading in data in chunks
loading file: 7chunks [00:09,  1.31s/chunks]                   
INFO:gtfparse:swifter found, processing in parallel
INFO:gtfparse:Repairing non-standard 'attributes'


INFO:gtfparse:Converting non-integer 'start' values to 0


INFO:gtfparse:Converting non-integer 'end' values to 0


In [6]:
annot_df

,seqname,source,feature,start,end,score,strand,frame,attribute
0,ENST00000618881,tappAS,transcript,1,3193,NaN,-,0,ID=ENST00000618881;primary_class=protein_codin...
1,ENST00000618881,tappAS,gene,1,3193,NaN,-,0,ID=NXF2B;Name=NXF2B;Desc=nuclear RNA export fa...
2,ENST00000618881,tappAS,CDS,1670,2896,NaN,-,0,ID=Q5JRM6;Desc=ENSP00000482525;PosType=T
3,ENST00000618881,UTRsite,PAS,3168,3193,NaN,-,0,ID=U0043;Name=PAS;Desc=Polyadenylation Signal;...
4,ENST00000618881,UTRsite,uORF,28,159,NaN,-,0,ID=U0033;Name=uORF;Desc=Upstream Open Reading ...
...,...,...,...,...,...,...,...,...,...
6398377,ENST00000335367,UniProtKB/Swiss-Prot_Phosphosite,PTM,539,539,NaN,NaN,0,ID=Phosphoserine;Name=Phosphoserine;Desc=Phosp...
6398378,ENST00000335367,UniProtKB/Swiss-Prot_Phosphosite,PTM,337,337,NaN,NaN,0,ID=Phosphoserine;Name=Phosphoserine;Desc=Phosp...
6398379,ENST00000335367,UniProtKB/Swiss-Prot_Phosphosite,BINDING,414,417,NaN,NaN,0,ID=Np_Bind;Name=Np_Bind;Desc=Gtp_SwissProt;Pos...
6398380,ENST00000335367,UniProtKB/Swiss-Prot_Phosphosite,PTM,552,552,NaN,NaN,0,ID=Phosphoserine;Name=Phosphoserine;Desc=Phosp...


In [7]:
dcTrans = annot_df[annot_df['source'] == "tappAS"][annot_df["feature"].isin(["transcript","gene","CDS"])]

/home/milo/miniconda3/envs/sqanti3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [18]:
len(dcTrans["seqname"].unique())

188667

In [9]:
dcExon = annot_df[annot_df["feature"] == "exon"]
len(dcExon['seqname'].unique())

188667

In [10]:
dcGenomic = annot_df[annot_df["feature"] == "genomic"]
len(dcGenomic['seqname'].unique())

188667

In [11]:
dcSpliceJunctions = annot_df[annot_df["feature"] == "splice_junction"]
len(dcSpliceJunctions['seqname'].unique())

163788

In [12]:
dcProt = annot_df[annot_df["feature"] == "protein"]
len(dcProt['seqname'].unique())

85103

In [13]:
dcTranscriptAttributes = annot_df[annot_df["source"] == "TranscriptAttributes"]
len(dcTranscriptAttributes['seqname'].unique())

0

In [16]:
dcTransFeatures = annot_df[annot_df["attribute"].str.contains("PosType=T")][~annot_df["feature"].isin(['exon',"genomic","splice_junction","protein","transcript","gene","CDS"])]
len(dcTransFeatures['seqname'].unique())

/home/milo/miniconda3/envs/sqanti3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


124947

In [17]:
dcProtFeatures = annot_df[annot_df["attribute"].str.contains("PosType=[P|G|N]")][~annot_df["feature"].isin(['exon',"genomic","splice_junction","protein","transcript","gene","CDS"])]
len(dcProtFeatures['seqname'].unique())

/home/milo/miniconda3/envs/sqanti3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


79641

In [21]:
dcProtFeatures.groupby("seqname")